# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Multiband-Resting-State-fMRI" data-toc-modified-id="Multiband-Resting-State-fMRI-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Multiband Resting-State fMRI</a></div><div class="lev2 toc-item"><a href="#Summary" data-toc-modified-id="Summary-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Summary</a></div><div class="lev2 toc-item"><a href="#Methodology" data-toc-modified-id="Methodology-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Methodology</a></div><div class="lev1 toc-item"><a href="#Measure-Dynamic-Functional-Connectivity" data-toc-modified-id="Measure-Dynamic-Functional-Connectivity-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Measure Dynamic Functional Connectivity</a></div><div class="lev2 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2 toc-item"><a href="#Design-and-Run-DyNe-Pipeline" data-toc-modified-id="Design-and-Run-DyNe-Pipeline-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Design and Run DyNe Pipeline</a></div><div class="lev3 toc-item"><a href="#Plot-Sample-BOLD-Signal" data-toc-modified-id="Plot-Sample-BOLD-Signal-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Plot Sample BOLD Signal</a></div><div class="lev3 toc-item"><a href="#Generate-proc_list" data-toc-modified-id="Generate-proc_list-232"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Generate proc_list</a></div><div class="lev3 toc-item"><a href="#Echobase-Pipeline-for-adajcency-matrices" data-toc-modified-id="Echobase-Pipeline-for-adajcency-matrices-233"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Echobase Pipeline for adajcency matrices</a></div><div class="lev3 toc-item"><a href="#Run-the-Pipeline" data-toc-modified-id="Run-the-Pipeline-234"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>Run the Pipeline</a></div><div class="lev2 toc-item"><a href="#Plot-Adjacency-Matrices" data-toc-modified-id="Plot-Adjacency-Matrices-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Plot Adjacency Matrices</a></div><div class="lev2 toc-item"><a href="#Verify-Dynamic-Functional-Connectivity" data-toc-modified-id="Verify-Dynamic-Functional-Connectivity-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Verify Dynamic Functional Connectivity</a></div>

# Multiband Resting-State fMRI

## Summary

*__Introduction__*

  * The brain is a complex organ that manifests a network of interactions, at the finer scale, between neurons and neuron populations, and at the meso-scale, between large functional domains. Over time, the pattern of interactions between brain regions change, describing a dynamic system capable of supporting cognition during tasks and at rest.
  
  * Recent advances in functional brain imaging make it possible to acquire images from multiple brain slices simultaneously, known as multiband imaging. 
  
  * Multiband imaging improves the sampling frequency of the blood oxygenation level dependant (BOLD) signal from $\approx$0.5 Hz to $\approx$2.0 Hz. With faster sampling of the BOLD signal, we are able to examine functional interactions across whole brain at unprecendent temporal resolution.
  
  * Previous work has indicated that functional associations between low-frequency components of the fMRI signal (0–0.15 Hz) can be attributed to task-related functional connectivity, whereas associations between high-frequency components (0.2–0.4 Hz) cannot. This frequency specificity of task-relevant functional connectivity is likely to be due at least in part to the hemodynamic response function, which might act as a noninvertible bandpass filter on underlying neural activity.
  
*__References__*
  - Pan, Wen-Ju, et al. "Infraslow LFP correlates to resting-state fMRI BOLD signals." Neuroimage 74 (2013): 288-297.
  - F. T. Sun, L. M. Miller, and M. D’Esposito, “Measuring interregional functional connectivity using coherence and partial coherence analyses of fMRI data.” Neuroimage 21 (2004): 647–658.



## Methodology

*__Multi-Taper Coherence__*

  * Multi-Taper methods are designed to limit the leakage of spectral power from other frequency bands into the frequency bands of interest. This is a critical concern when spectral properties of the signal are computed in discrete temporal windows. Different shapes of the temporal window can cause unwanted effects in the Fourier domain.

  * The choice of window length and shape impacts the allowable frequency resolution of the signal power spectrum. Slepian sequences, or orthogonal, wave-like eigenfunctions, give the optimal windowing to achieve a spectral bandwidth resolution on the interval $[-W, W]$. To achieve this resolution, multiple sequences are required, such that the optimal number of Slepian sequences, $k\approx2WT$, where $k$ is the number of Slepian sequences, $2W$ is the desired spectral resolution bandwidth, and $T$ is the duration of the window.
  
  * Suppose we wanted to estimate dynamic functional connectivity of fMRI in 30 second windows between 0.034 Hz and 0.167 Hz (i.e. a bandwidth resolution of 0.133 Hz), then the time-bandwidth product would be $4.0 = 0.133 Hz * 30 sec$, and the optimal number of tapers would be $2*4.0-1 = 7$.
  
   

# Measure Dynamic Functional Connectivity

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/ValueLearning/Glasser180/wGSregress'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_ExpData = path_PeriphData + '/e01c-Dyne_FuncNetw'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)
        
if not os.path.exists('./e01c-Figures'):
    os.makedirs('./e01c-Figures')

## Generate List of Data

In [ ]:
subj_ids = [full_subj_path.split('/')[-1]
         for full_subj_path in glob.iglob('{}/*'.format(path_CoreData))]
run_ids = ['day1', 'day2', 'day3', 'day4']

subj_dict = {}
for subj_id in subj_ids:
    subj_dict[subj_id] = []
    
    for run_id in run_ids:
        run_path = '{}/{}/{}/REST/TS_Glasser180.csv'.format(path_CoreData, subj_id, run_id)
        subj_dict[subj_id].append(run_path)

## Design and Run DyNe Pipeline

### Plot Sample BOLD Signal

In [ ]:
#%matplotlib inline

for subj_id in subj_dict.keys():
    for run_path in subj_dict[subj_id]:
                    
        df = pd.read_csv(run_path)
        norm_df = (df - df.mean(axis=0)).as_matrix()
        norm_df = norm_df / (0.75*np.std(norm_df, axis=0))
        
        n_samp = df.shape[0]
        Fs = 2.0
        time = 120
        samples = np.arange(0, time*Fs)
        
        # Construct result figure
        plt.figure()
        ax = plt.subplot(111)
        
        ch_subset = np.arange(0, df.shape[1], 8)
        for ch in ch_subset:
            ax.plot(samples/Fs, ch + norm_df[map(int, samples), ch], color=[66/256., 152/256., 221./256])
            #ax.plot(time, ch + norm_df[:, ch]/(0.75*norm_df[:, ch].max()), )
            #break

        ax.set_ylim([ch_subset.min()-4, ch_subset.max()+4])

        # Axis Settings
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        ax.set_xticks([0, 30, 60, 90, 120])
        ax.set_ylim([ch_subset.min()-6, ch_subset.max()+6])
        ax.set_ylabel('Brain Regions')
        ax.set_xlabel('Time (sec)')
        
        plt.savefig('./e01c-Figures/BOLD_Signal.svg')
        plt.show()
        break
    break

### Generate proc_list

In [ ]:
proc_list = []

for subj_id in subj_dict.keys():
    for run_id, run_path in zip(run_ids, subj_dict[subj_id]):
        
        # Generate path to the input fMRI clip
        adj_path = '{}/Adjacency.{}.{}.npz'.format(path_ExpData,
                                                   subj_id, run_id)
        
        if not os.path.exists(run_path):
            print('Procesing item: {}, not found'.format(run_path))
            continue
            
        proc_list.append({'bold_path': run_path,
                          'adj_path': adj_path})

### Echobase Pipeline for adajcency matrices

In [ ]:
# Parameters
params = {'signal': {},
          'coherence': {}}

# Signal processing
params['signal'] = {'fs': 2,
                    'win_len': 30.0,
                    'win_disp': 30.0}

# Coherence measurement
params['coherence'] = {'time_band': 2.5,
                       'n_taper': 4,
                       'cf': [0.034, 0.20]}


def measure_coherence(proc_item):
    print('\nProcessing: {}'.format(proc_item['bold_path']))
    
    # Load the BOLD Signal from CSV
    df = np.array(np.genfromtxt(proc_item['bold_path'], delimiter=','),
                  dtype=np.float)
    
    n_sample, n_node = df.shape
    
    
    # Check window size and displacement
    n_win_len = int(params['signal']['win_len'] * params['signal']['fs'])
    n_win_disp = int(params['signal']['win_disp'] * params['signal']['fs'])
    if n_win_len > n_sample:
        raise ValueError('win_len cannot be longer than signal duration')
    n_wins = int((n_sample - n_win_len) /
                 n_win_disp) + 1
            
    # Compute dynamic functional connectivity
    # Initialize a time-varying adjacency matrix
    adj_matr = np.zeros((n_wins, n_node, n_node))
    for win_id, idx in enumerate(range(0, n_wins * n_win_disp, n_win_disp)):
        win = df[idx:idx+n_win_len, :]
        adj_matr[win_id, ...] = Echobase.Network.Functional.coherence.multitaper(win, params['signal']['fs'],
                                                                                 **params['coherence'])
    # Save the output
    np.savez(proc_item['adj_path'],
             adj_matr=adj_matr)

### Run the Pipeline

In [ ]:
from multiprocessing import Pool
parallel_run = True    
    
if parallel_run:
    mp = Pool(7)
    mp.map(measure_coherence, proc_list)
else:
    map(measure_coherence, proc_list)

## Plot Adjacency Matrices

In [ ]:
# Get list of Output Files
adj_paths = glob.glob('{}/Adjacency.*.npz'.format(path_ExpData))
adj_path = np.random.permutation(adj_paths)[0]

# Figure
%matplotlib inline
for ix, ii in enumerate([0, 1, -1]):
       
    df = np.load(adj_path)
    data = df['adj_matr'][ix, :, :]
    df.close()

    plt.figure()
    ax = plt.subplot(111)
    mat = ax.matshow(data, cmap='viridis', vmin=0, vmax=1)
    ax.set_axis_off()
    
    plt.savefig('./e01-Figures/Sample_Adjacency_{}.svg'.format(ix+1))
    plt.show()
    plt.close()  

## Verify Dynamic Functional Connectivity

In [ ]:
n_subj = len(subj_ids.keys())
n_date = len(subj_ids[subj_ids.keys()[0]])

var_edge_dyn = np.zeros((n_subj, n_date))
var_edge_null = np.zeros((n_subj, n_date))

for s_i, subj_id in enumerate(subj_ids.keys()):
    for d_i, date_id in enumerate(subj_ids[subj_id]):
        
        # Load the data
        df = np.load('{}/Adjacency.{}.{}.npz'.format(path_ExpData, subj_id, date_id))
        
        # Compute the standard deviation of the dynamic edge fluctuation
        adj_matr = df['adj_matr']
        cfg_matr = convert_adj_matr_to_cfg_matr(adj_matr)
        var_edge_dyn[s_i, d_i] = np.mean(np.var(cfg_matr, axis=0))
        
        # Compute the standard deviation of the surrogate, static adjacency matrices
        adj_surr_matr = df['adj_surr_matr']
        cfg_surr_matr = convert_adj_matr_to_cfg_matr(adj_surr_matr)
        var_edge_null[s_i, d_i] = np.mean(np.var(cfg_surr_matr, axis=0))
        
# Figure
%matplotlib inline

plt.figure()
ax = plt.subplot(111)
bplot = ax.boxplot([var_edge_dyn.mean(axis=1),
                    var_edge_null.mean(axis=1)],
                   patch_artist=True);
Echobase.Plotting.fig_format.set_box_color(bplot, 'k', [[0.00, 0.37, 1.00],
                                                        [0.75, 0.75, 0.75]])


# Axis Settings
ax.set_xlim([0, 3])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Mean Variance of Edge Strength');
ax.set_xticklabels(['Windowed\nFunctional Connections', 'Static Surrogate\nFunctional Connections']);

plt.savefig('./e01-Figures/Dynamic_Static_FuncConn.svg')
plt.show()
plt.close()              